In [1]:
import json
import numpy as np

path = "../tweet_data/"

files = ["tweets_#gohawks.txt", "tweets_#gopatriots.txt", \
        "tweets_#nfl.txt", "tweets_#patriots.txt", \
        "tweets_#sb49.txt", "tweets_#superbowl.txt"]

for i, fl in enumerate(files):
    print "files[" + str(i) + "] => " + fl

files[0] => tweets_#gohawks.txt
files[1] => tweets_#gopatriots.txt
files[2] => tweets_#nfl.txt
files[3] => tweets_#patriots.txt
files[4] => tweets_#sb49.txt
files[5] => tweets_#superbowl.txt


In [2]:
# transform the 2015 Feb. 1, 8:00 am and 8:00 pm into timestamp form
import datetime, time
import pytz

start_time = time.mktime(time.strptime("2015-02-01 08:00:00",'%Y-%m-%d %H:%M:%S'))
end_time = time.mktime(time.strptime("2015-02-01 20:00:00",'%Y-%m-%d %H:%M:%S'))
# used as the zero point of the time
base_time = time.mktime(time.strptime("2015-01-01 00:00:00",'%Y-%m-%d %H:%M:%S'))

In [3]:
def floor_date(date):
    return date // 3600

In [4]:
# the extracted data contains
# 0. the citation date 1. the favourite_count 2. the followers number
# 3. the length of the title 4. the number of the twitters
def load_file(file):
    '''
    load the time, follows, and retweets
    '''
    # 0. before Feb. 1, 8:00 a.m. 1. between 2. after 8:00 p.m.
    result = [[], [], []]
    for line in open(path + file, 'r') :
        tmp = []
        a = json.loads(line)
        citation_date = a['citation_date']
        tmp.append(int((citation_date - base_time) // 3600))
        tmp.append(a['author']['followers'])
        tmp.append(a['tweet']['user']['favourites_count'])
        tmp.append(len(a['title']))
        tmp.append(1)
        if citation_date < start_time:
            result[0].append(tmp)
        elif citation_date < end_time:
            result[1].append(tmp)
        else:
            result[2].append(tmp)
    return result

In [5]:
def group_data(dataset):
    '''
    parameter dataset is one of the three time slot defined above, namely, data[0], data[1], data[2]
    count all the features in 1-hour slot
    
    return the train feature (train_set[0:4]) and the result (train_set[5])
    '''
    # get the maximum and minimum time
    max_time = dataset[0][0]
    min_time = dataset[0][0]
    for p in dataset:
        time = p[0]
        if max_time < time:
            max_time = time
        if min_time > time:
            min_time = time
    
    train_set = []
    
    for i in range(min_time, max_time + 1):
        train_set.append([0] * 5)
    
    for p in dataset:
        index = p[0] - min_time
        train_set[index][0] = p[0]
        for i in range(1, 5):
            train_set[index][i] += p[i]
    
    result = []
    for i in range(0, max_time - min_time):
        result.append(train_set[i + 1][4])
    
    return train_set[:-1], result

In [6]:
def get_error(predict, target):
    '''
        used to get the average error between predict and target
    '''
    length = len(target)
    return sum([abs(predict[i] - target[i]) for i in range(length)]) / float(length)

In [7]:
# using k-fold to divide the train and test set 
from sklearn.model_selection import KFold

n_splits = 10
kf = KFold(n_splits = n_splits, shuffle = True)

def k_fold(X, y, train_model):
    '''
        use k-fold to split the data and use train_model to train and predict
        return the average error of the model
    '''
    error = 0
    for train_index, test_index in kf.split(X):
        train_feature = []
        test_feature = []
        train_result = []
        test_result = []
        for index in train_index:
            train_feature.append(X[index])
            train_result.append(y[index])
        for index in test_index:
            test_feature.append(X[index])
            test_result.append(y[index])
            
        model = train_model.fit(train_feature, train_result)
        predict = model.predict(test_feature)
        error += get_error(predict, test_result)
    return error / n_splits

In [8]:
%%capture
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR

lr = linear_model.LinearRegression()
clf = MLPRegressor(alpha = 0.001, hidden_layer_sizes = (500,), \
                           activation = 'tanh', verbose = 'True', learning_rate = 'adaptive')
svm = LinearSVR(random_state = 0)

In [9]:
%%capture
time_range = {0: "Before Feb. 1, 8:00 a.m. ", 1: "Between Feb. 1, 8:00 a.m. and 8:00 p.m. ", \
              2: "After Feb. 1, 8:00 p.m."}
final_result = []
for f in files:
    file_data = load_file(f)
    for i in range(3):
        X, y = group_data(file_data[i])
        a = "the file is " + f + " , the time range is " + time_range[i] + \
        " and the error is \n" + str(k_fold(X, y, lr)) + \
        " and \n" + str(k_fold(X, y, clf)) + \
        " and \n" + str(k_fold(X, y, svm))
        final_result.append(a)

In [10]:
for x in final_result:
    print x

the file is tweets_#gohawks.txt , the time range is Before Feb. 1, 8:00 a.m.  and the error is 
303.639361865003 and 
227.77633640162193 and 
197.7600882506802
the file is tweets_#gohawks.txt , the time range is Between Feb. 1, 8:00 a.m. and 8:00 p.m.  and the error is 
7971.9156983906 and 
5452.357298902855 and 
4284.136875145707
the file is tweets_#gohawks.txt , the time range is After Feb. 1, 8:00 p.m. and the error is 
35.44609398937148 and 
37.6777523638481 and 
113.85460925468765
the file is tweets_#gopatriots.txt , the time range is Before Feb. 1, 8:00 a.m.  and the error is 
15.085538651280505 and 
13.966260956085554 and 
45.5720173729588
the file is tweets_#gopatriots.txt , the time range is Between Feb. 1, 8:00 a.m. and 8:00 p.m.  and the error is 
813.886118499752 and 
1471.8396477097172 and 
819.538813078666
the file is tweets_#gopatriots.txt , the time range is After Feb. 1, 8:00 p.m. and the error is 
3.8792945798948453 and 
3.5654366445156596 and 
5.665022138169254
the f

In [18]:
%%capture
final_result = []

# load the file and aggregate all the data
aggregate_file = [[], [], []]
for f in files:
    file_data = load_file(f)
    for i in range(len(aggregate_file)):
        aggregate_file[i] += file_data[i]

for i in range(3):
    X, y = group_data(file_data[i])
    a = "the time range is " + time_range[i] + \
    " and the error is \n" + str(k_fold(X, y, lr)) + \
    " and \n" + str(k_fold(X, y, clf)) + \
    " and \n" + str(k_fold(X, y, svm))
    final_result.append(a)

In [16]:
for i in final_result:
    print i

the time range is Before Feb. 1, 8:00 a.m.  and the error is 
253.27259234699687 and 
411.36714482239296 and 
1080.5463566482615
the time range is Between Feb. 1, 8:00 a.m. and 8:00 p.m.  and the error is 
115980.00756452112 and 
87689.34303384952 and 
85851.11489502201
the time range is After Feb. 1, 8:00 p.m. and the error is 
175.92343755502975 and 
711.7407115256639 and 
385.96078201990895
